# Background Information
Reference Materials and codes 
 - https://appliedsciences.nasa.gov/sites/default/files/2023-11/SAR_Part3.pdf - Background information on SAR and SAR properties
 - https://code.earthengine.google.com/f5c2f984c053c8ea574bfcd4040d084e - UNSPIDER Code for flood extent mapping adapted in this code
 - https://github.com/UN-SPIDER/radar-based-flood-mapping?tab=readme-ov-file - GitHub Code for flood analysis
 - https://colab.research.google.com/github/UN-SPIDER/radar-based-flood-mapping/blob/main/resources/notebooks/radar-based-flood-mapping-colab.ipynb#scrollTo=wvsKF_MmW3_U - Google Colab Notebook for flood analysis

## Start with the installation of the python libraries to support your analysis

In [1]:
try:
    import ee
#     %pip install earthengine-api --upgrade
    ee.Authenticate()
    ee.Initialize()
except:
#     %pip install earthengine-api --upgrade
    import ee
    ee.Authenticate()
    ee.Initialize()

try:
    import geemap.foliumap as geemap
except:
#     %pip install geemap
    import geemap.foliumap as geemap

## Roadmap for the analysis
1. Define the date of flooding
2. Define your area of interest where the flooding happened
3. Use the dates to get images before and after the floods
4. Analyze the difference between the two images to get flooded areas
5. Improve the SAR derived flood maps by removing permanent areas, high slopes and adjacent pixels
6. Visualize your map

## Advanced Analysis
1. Analyze exposed population
2. Agricultural areas affected
3. Urban areas

## 1. Define the dates for the floods

In [2]:
## Date of flood 2019-07-04
before_start = '2019-06-25'
before_end = '2019-07-05'

after_start = '2019-07-05'
after_end = '2019-07-15'

## 2. Area of interest

In [3]:
## Get your area of interest by uncommenting the next code
## Draw a polygon 
## click on the drawn polygon
## copy and paste the coordinates in the next line of code
# aoimap = geemap.Map()
# aoimap

In [4]:
aoi_ee = ee.Geometry.Polygon([[-0.552207,5.429763],[-0.552207,5.886368],[0.079372,5.886368],[0.079372,5.429763],[-0.552207,5.429763]])

## 3. Select our images

In [5]:
polarization = 'VH'
pass_direction = 'ASCENDING'
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode','IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters',10)) \
    .filterBounds(aoi_ee) \
    .select(polarization)

In [6]:
before_collection = collection.filterDate(before_start, before_end) # collection before flood
after_collection = collection.filterDate(after_start,after_end) # collection after floods

In [7]:
before_count = before_collection.size()
print(before_count.getInfo(), ' Number of Images Before the flood')
after_count = after_collection.size()
print(after_count.getInfo(), ' Number of Images after the flood')

2  Number of Images Before the flood
1  Number of Images after the flood


## 4. Analyze the Data

In [8]:
# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi_ee)
after = after_collection.mosaic().clip(aoi_ee)

In [9]:
# Apply reduce the radar speckle by smoothing  
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

In [10]:
threshold = 0.8 # This is by trial and error for before and after images
difference = after_filtered.divide(before_filtered) # Calculate the difference between the two images
difference_binary = difference.lt(threshold)

## 5. Improve on the outputs

### 5.1 Remove permanent water bodies

In [20]:
swater = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('seasonality') # Surface water dataset
swater_mask = swater.gte(10).updateMask(swater.gte(10)) # mask where there is water > 10 months of the year
flooded_mask = difference_binary.where(swater_mask,0) # mask perennial waters = 0
flooded = flooded_mask.updateMask(flooded_mask)

### 5.2 Remove isolated pixels

In [21]:
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

### 5.3 Maintain pixels that are within 5% slope

In [22]:
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope').clip(aoi_ee)
flooded = flooded.updateMask(slope.lt(5))

## 6 Export the output to your local machine

In [14]:
## If you would like to export the output to your local machine, do uncomment the next line

#geemap.ee_export_image(ee_object=flooded, filename="floodedAreas.tif", scale=30, crs=None, region=aoi_ee, file_per_band=False)

## Using GEE, calculate the exposure levels

### Population Affected

In [15]:
population_count = ee.Image('JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015').clip(aoi_ee) # Pop Global Data
population_exposed = population_count \
  .updateMask(flooded) \
  .updateMask(population_count) 

stats = population_exposed.reduceRegion( \
            reducer= ee.Reducer.sum(), \
            geometry= population_exposed.geometry(), \
            scale= 250, \
            maxPixels = 1e9 \
            )
number_pp_exposed = stats.getNumber('population_count').round().getInfo()

print(number_pp_exposed, ' people are affected')

4162  people are affected


### Agricultural area affected

In [23]:
LC = ee.ImageCollection('MODIS/061/MCD12Q1') \
        .filterDate('2014-01-01',after_end) \
        .sort('system:index',False) \
        .select("LC_Type1") \
        .first() \
        .clip(aoi_ee) 

In [24]:
## get cropland pixels using the classes cropland (>60%) and Cropland/Natural 
## Vegetation Mosaics: mosaics of small-scale cultivation 40-60% incl. natural vegetation
cropmask = LC.eq(14).And(LC.eq(14))
cropland = LC.updateMask(cropmask)
  
## Get MODIS projection
MODISprojection = LC.projection() 

## Reproject flood layer to MODIS scale
flooded_res = flooded \
    .reproject( \
    crs = MODISprojection \
  )

## Calculate affected cropland using the resampled flood layer
cropland_affected = flooded_res.updateMask(cropland)

## get pixel area of affected cropland layer
crop_pixelarea = cropland_affected.multiply(ee.Image.pixelArea()) ## calcuate the area of each pixel 

## sum pixels of affected cropland layer
crop_stats = crop_pixelarea.reduceRegion( \
                reducer = ee.Reducer.sum(), \
                geometry = aoi_ee, \
                scale = 500, \
                maxPixels = 1e9 \
)
  
## convert area to hectares
crop_area_ha = crop_stats \
  .getNumber(polarization) \
  .divide(10000) \
  .round() \
  .getInfo()
crop_area_ha

0

## Affected Urban Area

In [25]:
## Using the same MODIS Land Cover Product 
##Filter urban areas
urbanmask = LC.eq(13)
urban = LC.updateMask(urbanmask)

## Calculate affected urban areas using the resampled flood layer
urban_affected = urban.mask(flooded_res).updateMask(urban)

## get pixel area of affected urban layer
urban_pixelarea = urban_affected \
    .multiply(ee.Image.pixelArea()) ##calcuate the area of each pixel 

## sum pixels of affected cropland layer
urban_stats = urban_pixelarea.reduceRegion( \
                reducer= ee.Reducer.sum(),\
                geometry= aoi_ee, \
                scale= 500, \
                bestEffort= True, \
)

## convert area to hectares
urban_area_ha = urban_stats \
  .getNumber('LC_Type1') \
  .divide(10000) \
  .round()

## 7. Visualize the Output

In [26]:
Map = geemap.Map()
Map.centerObject(aoi_ee, 11)
Map.addLayer(before_filtered, {min:-25,max:0}, 'Before Flood',0)
Map.addLayer(after_filtered, {min:-25,max:0}, 'After Flood',1)
Map.addLayer(difference,{min:0,max:2},"Difference Layer",0)
Map.addLayer(flooded,{'palette':"0000FF"},'Flooded areas')
Map.addLayer(population_count, {min: 0,max: 200.0,'palette':['060606','337663','337663','ffffff']}, 'Population Density',0)
Map.addLayer(aoi_ee, {}, 'Layer name') ## Visualize your shapefile in Google Earth Engine
Map.addLayer(before, {"min": -25, "max":0}, "before")
Map.addLayer(after, {"min": -25, "max":0}, "after")
Map.addLayer(difference, {"min": 0.4, "max":0.7}, "difference")
Map.addLayer(flooded, {"palette":["0000FF"]}, "floods")
Map